## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import json
import requests
import pandas as pd
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://services3.arcgis.com/ibgDyuD2DLBge82s/arcgis/rest/services/Fresno_County_Zip_Data_Summary/FeatureServer/0/query?f=json&where=FZIP%3C%3E%2799999%27%20AND%20COUNT_Cases%3E%3D75%20AND%20FZIP%20IS%20NOT%20NULL&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&groupByFieldsForStatistics=FCITY&orderByFields=value%20desc&outStatistics=%5B%7B%22statisticType%22%3A%22sum%22%2C%22onStatisticField%22%3A%22COUNT_Cases%22%2C%22outStatisticFieldName%22%3A%22value%22%7D%5D&resultType=standard&cacheHint=true"

In [7]:
r = requests.get(url)

In [8]:
data = r.json()

## Parse

In [9]:
dict_list = []

In [10]:
for item in data["features"]:
    d = dict(
        county="Fresno",
        area=item["attributes"]["FCITY"],
        confirmed_cases=item["attributes"]["value"],
    )
    dict_list.append(d)

In [11]:
df = pd.DataFrame(dict_list)

In [12]:
df["area"] = df["area"].str.title()

Get timestamp

In [14]:
date_url = "https://services3.arcgis.com/ibgDyuD2DLBge82s/arcgis/rest/services/Fresno_County_Zip_Data_Summary/FeatureServer/0/?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [15]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [16]:
tz = pytz.timezone("America/Los_Angeles")

In [17]:
latest_date = datetime.fromtimestamp(timestamp / 1000, tz).date()

In [18]:
df["county_date"] = latest_date

## Trim

In [19]:
df["county"] = "Fresno"

In [20]:
df["area"] = df["area"].str.title()

In [21]:
c = df[["county", "area", "county_date", "confirmed_cases"]]

## Vet

In [25]:
try:
    assert not len(df) > 30
except AssertionError:
    raise AssertionError("Fresno County's place scraper has additional rows")

In [26]:
try:
    assert not len(df) < 30
except AssertionError:
    raise AssertionError("Fresno County's place scraper is missing rows")

## Export

Mark the current date

In [27]:
tz = pytz.timezone("America/Los_Angeles")

In [28]:
today = datetime.now(tz).date()

In [29]:
slug = "fresno"

In [30]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [31]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [32]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [33]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [34]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)

In [36]:
df[df.area == "Fresno"]

,county,area,county_date,confirmed_cases,date
2634,Fresno,Fresno,NaT,128,2020-04-14 00:00:00
2618,Fresno,Fresno,NaT,148,2020-04-15 00:00:00
2602,Fresno,Fresno,NaT,159,2020-04-16 00:00:00
2586,Fresno,Fresno,NaT,166,2020-04-17 00:00:00
2570,Fresno,Fresno,NaT,189,2020-04-20 00:00:00
...,...,...,...,...,...
0,Fresno,Fresno,2021-08-02,61638,2021-08-22
0,Fresno,Fresno,2021-08-02,61638,2021-08-23
0,Fresno,Fresno,2021-08-02,61638,2021-08-24
0,Fresno,Fresno,2021-08-02,61638,2021-08-25
